# 🏥 고양이 의료 문서 분석

이 노트북은 `data/은내과.pdf` 파일을 Tesseract로 분석하여 의료 데이터를 추출합니다.

## 주요 기능
- PDF → 이미지 변환
- Tesseract를 통한 테이블 데이터 추출
- 의료 검사 결과 구조화
- JSON 형태로 데이터 저장

## 1. 환경 설정 및 라이브러리 로드

In [ ]:
# 시스템 패키지 업데이트
# sudo apt-get update

# Tesseract OCR 및 한국어 언어팩 설치
# sudo apt-get install tesseract-ocr tesseract-ocr-eng tesseract-ocr-kor

In [10]:
# 필요한 라이브러리 설치 (첫 실행 시만)
!pip install pytesseract

In [ ]:
# 설치 확인
# tesseract --version
# tesseract --list-langs
# echo $TESSDATA_PREFIX

## 2. 파일 로드

In [12]:
import io
from typing import List, Dict, Any

class UploadedFileManager:
    def __init__(self):
        # 업로드된 파일들이 저장되는 리스트
        # self.files 의 구조 예시:
        # [
        #     {
        #         "name": "blood_test1.png",       # 파일 이름 (str)
        #         "content": b"...",               # 원본 파일 내용 (bytes)
        #         "io": <_io.BytesIO object>       # BytesIO 객체 (PIL/OpenCV 사용 가능)
        #     },
        #     {
        #         "name": "blood_test2.jpg",
        #         "content": b"...",
        #         "io": <_io.BytesIO object>
        #     }
        # ]
        self.files: List[Dict[str, Any]] = []

    def handler(self, change) -> None:
        """
        ipywidgets FileUpload 위젯의 observe 이벤트 핸들러.

        Args:
            change (dict): ipywidgets 가 observe 이벤트에서 전달하는 dict
                change 의 구조 예시:
                {
                    'name': 'value',                # 변경된 속성 이름
                    'old': (),                      # 이전 값 (tuple of UploadedFile)
                    'new': (                        # 새로운 값 (tuple of UploadedFile)
                        UploadedFile(
                            name='blood_test1.png',
                            type='image/png',
                            size=12345,
                            content=b'...'
                        ),
                        UploadedFile(
                            name='blood_test2.jpg',
                            type='image/jpeg',
                            size=23456,
                            content=b'...'
                        )
                    ),
                    'owner': FileUpload(...)        # 이벤트를 발생시킨 위젯 객체
                }

        Returns:
            None: 반환값은 없으며, self.files 에 업로드된 파일 리스트를 저장합니다.
        """
        try:
            if not change or "new" not in change or not change["new"]:
                self.files = []
                return

            files = list(change["new"])  # tuple of UploadedFile → list 변환

            self.files = [
                {
                    "name": f.name,
                    "content": f.content,
                    "io": io.BytesIO(f.content)
                }
                for f in files
            ]

            print("✅ 업로드 완료:", [f["name"] for f in self.files])

        except Exception as e:
            print(f"❌ 파일 처리 오류: {e}")
            self.files = []

In [19]:
import ipywidgets as widgets
from IPython.display import display

# 1. FileUpload 위젯 생성
uploader = widgets.FileUpload(accept='.png,.jpg,.jpeg', multiple=True)
display(uploader)

# 2. 파일 매니저 인스턴스 생성
file_manager = UploadedFileManager()

# 3. observe 연결
uploader.observe(file_manager.handler, names="value")

FileUpload(value=(), accept='.png,.jpg,.jpeg', description='Upload', multiple=True)

In [38]:
import numpy as np, cv2
import pytesseract
from pytesseract import Output
import pandas as pd

# Tesseract 설정
TESS_LANG = "kor+eng"
TESS_CONFIG = "--oem 3 --psm 6"  # 일반 문서에 적합, 표는 6/11/12/13 시도
# 숫자·소수점 강화를 위한 별도 설정
TESS_CONFIG_NUMERIC = f"{TESS_CONFIG} -c load_system_dawg=0 -c load_freq_dawg=0 -c tessedit_char_whitelist=0123456789.,-%"

def run_tesseract_ocr(img, lang: str = TESS_LANG, config: str = TESS_CONFIG):
    """
    Tesseract 결과를 사람이 보기 좋은 포맷으로 반환
    - df: 단어 단위 전체 결과(DataFrame)
    - lines: 줄 단위 요약 리스트([{text, score, bbox, meta}...])
    - words: 단어 단위 리스트([{text, score, bbox, meta}...])
    """
    # Tesseract 실행 (단어 레벨)
    raw = pytesseract.image_to_data(img, lang=lang, config=config, output_type=Output.DICT)
    df = pd.DataFrame(raw)

    # 정리
    df["text"] = df["text"].fillna("").str.strip()
    df["conf"] = pd.to_numeric(df["conf"], errors="coerce").fillna(-1)
    df = df[(df["text"] != "") & (df["conf"] >= 0)].copy()

    # 보조 좌표
    df["x1"] = df["left"].astype(int)
    df["y1"] = df["top"].astype(int)
    df["x2"] = (df["left"] + df["width"]).astype(int)
    df["y2"] = (df["top"] + df["height"]).astype(int)

    # 단어 리스트
    words = []
    for r in df.itertuples(index=False):
        bbox = [[int(r.x1), int(r.y1)], [int(r.x2), int(r.y1)], [int(r.x2), int(r.y2)], [int(r.x1), int(r.y2)]]
        words.append({
            "text": r.text,
            "score": float(r.conf) / 100.0,
            "bbox": bbox,
            "meta": {
                "page": int(r.page_num), "block": int(r.block_num), "para": int(r.par_num),
                "line": int(r.line_num), "word": int(r.word_num)
            }
        })

    # 줄 리스트(같은 line 그룹을 묶어 요약)
    group_cols = ["page_num", "block_num", "par_num", "line_num"]
    lines = []
    for keys, g in df.groupby(group_cols, sort=True):
        g_sorted = g.sort_values("word_num")
        text = " ".join(g_sorted["text"].tolist())
        x1, y1 = int(g["x1"].min()), int(g["y1"].min())
        x2, y2 = int(g["x2"].max()), int(g["y2"].max())
        bbox = [[x1, y1], [x2, y1], [x2, y2], [x1, y2]]
        score = float(g["conf"].mean()) / 100.0
        lines.append({
            "text": text,
            "score": score,
            "bbox": bbox,
            "meta": {"page": int(keys[0]), "block": int(keys[1]), "para": int(keys[2]), "line": int(keys[3])}
        })

    # 보기 편한 컬럼 순서로 정리된 DataFrame
    col_order = ["level","page_num","block_num","par_num","line_num","word_num","text","conf","left","top","width","height","x1","y1","x2","y2"]
    df = df[[c for c in col_order if c in df.columns]]

    return {"df": df, "lines": lines, "words": words}


In [43]:

# 4. 업로드 완료 후 파일 목록 접근
import json
print(json.dumps(file_manager.files, ensure_ascii=False, indent=2, default=str))


# 5. Tesseract OCR에 넘기기
import numpy as np, cv2


# 업로드된 모든 파일 순회
for f in file_manager.files:
    print(f"📂 Processing: {f['name']}")

    # 이미지 디코딩
    img_array = np.frombuffer(f["content"], np.uint8)
    cv_img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    # 흑백 변환
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)

    _, black = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 국소 대비 향상: 연한 글자 대비 강화
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)

    # 파란색 글자 보강 마스크
    # hsv = cv2.cvtColor(cv_img, cv2.COLOR_BGR2HSV)
    # lower_blue = np.array([90, 50, 50], dtype=np.uint8)   # 필요시 [85~95], [40~60] 등 미세 조정
    # upper_blue = np.array([130, 255, 255], dtype=np.uint8)
    # mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    # mask_blue = cv2.morphologyEx(mask_blue, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8), iterations=1)

    # 단순 이진화
    # _, thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)

    # 또는 적응형 이진화가 더 잘 먹힐 때가 많습니다
    # thresh = cv2.adaptiveThreshold(gray, 255,
    #                                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                                cv2.THRESH_BINARY, 25, 15)

    # 이진화 (가장자리 강조)
    # _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)

    # 이진화 (Otsu 사용: 소수점 보존에 유리) + 파란색 전경 보강 결합
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    # binary = cv2.bitwise_or(binary, mask_blue)  # 파란 글자는 전경(흰색)으로 유지

    # -------------------------------
    # 1. 수평 라인 제거
    # -------------------------------
    h_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    detected_lines_h = cv2.morphologyEx(binary, cv2.MORPH_OPEN, h_kernel, iterations=2)
    line_removed_h = cv2.subtract(binary, detected_lines_h)

    # -------------------------------
    # 2. 수직 라인 제거
    # -------------------------------
    v_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
    detected_lines_v = cv2.morphologyEx(line_removed_h, cv2.MORPH_OPEN, v_kernel, iterations=2)
    line_removed = cv2.subtract(line_removed_h, detected_lines_v)

    # -------------------------------
    # 3. 원래 색상으로 복원
    # -------------------------------
    processed = cv2.bitwise_not(line_removed)

    # 2x 스케일업(소숫점 보존)
    # scaled = cv2.resize(gray, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
    scaled = cv2.resize(processed, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)

    # 확인용 저장
    cv2.imwrite("debug_output/01_gray.png", gray)                # 흑백 이미지
    cv2.imwrite("debug_output/01_black.png", black)             # black
    cv2.imwrite("debug_output/02_binary.png", binary)            # 이진화 결과
    cv2.imwrite("debug_output/03_no_hlines.png", line_removed_h) # 수평선 제거 후
    cv2.imwrite("debug_output/04_no_vlines.png", line_removed)   # 수직선 제거 후
    cv2.imwrite("debug_output/05_processed.png", processed)      # 최종 OCR 입력 이미지
    cv2.imwrite("debug_output/06_scaled.png", scaled)            # 2x 스케일업

    # OCR 입력용 BGR 변환
    # ocr_input = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)
    ocr_input = cv2.cvtColor(scaled, cv2.COLOR_GRAY2BGR)

    # OCR 실행
    ocr_result = run_tesseract_ocr(ocr_input, lang=TESS_LANG, config=TESS_CONFIG) # 일반 패스(한/영 혼합)
    # ocr_result = run_tesseract_ocr(ocr_input, lang="eng", config=TESS_CONFIG_NUMERIC) # 숫자·소수점 강화

    # 확인용 출력: 줄 단위 상위 20개
    print("🔎 Lines (top 20 by reading order)")
    for i, row in enumerate(ocr_result["lines"][:20], start=1):
        print(f"[{i:02d}] conf={row['score']:.2f}  {row['text']}")

    # 확인용 출력: 단어 단위 상위 20개
    print("\n🔎 Words (top 20)")
    for i, w in enumerate(ocr_result["words"][:20], start=1):
        x1, y1 = w["bbox"][0]
        print(f"[{i:02d}] conf={w['score']:.2f} ({x1},{y1}) {w['text']}")

    # 노트북에서 표로 확인
    try:
        import pandas as pd
        with pd.option_context(
            'display.max_rows', None,
            'display.max_columns', None,
            'display.width', None,
            'display.max_colwidth', None
        ):
            display(ocr_result["df"])  # 전체 출력
        # 필요시 CSV로 저장하여 외부에서 확인
        # ocr_result["df"].to_csv("debug_output/ocr_result_full.csv", index=False)
    except Exception:
        pass

[
  {
    "name": "은내과 2.png",
    "content": "<memory at 0x72f959913a00>",
    "io": "<_io.BytesIO object at 0x72f9a7659300>"
  }
]
📂 Processing: 은내과 2.png
🔎 Lines (top 20 by reading order)
[01] conf=0.88  Date/Time : 2025-01-16 YS 1:51:10 Performed By [ProCyte_DX]
[02] conf=0.96  Name Unit Min Max Result
[03] conf=0.83  RBC 10x6/uL 6.54 12.2 573 LOW | a | | |
[04] conf=0.80  CT % 30.3 52.3 26.1 LOW | a | | |
[05] conf=0.74  4168 g/dL 9.8 16.2 78 Low | a | | |
[06] conf=0.76  Mev 개 35.9 53.1 45.5 NORMAL | | | | |
[07] conf=0.74  처 매 09 11.8 17.3 13.6 NORMAL | | | |
[08] conf=0.70  “MCHC g/dL 28.1 35.8 29.9 NORMAL | | 0 | |
[09] conf=0.69  RDW-CV % 15 27 337 HIGH | | | |!
[10] conf=0.62  -RETIC% % 0.2
[11] conf=0.71  -RETICH 10%3/|11Ｌ 3 50 10.3 NORMAL | | a | |
[12] conf=0.63  -RETIC-HGB 09 13.2 20.8 15 NORMAL | | 0 | |
[13] conf=0.78  “WBC 10x3/uL 2.87 17.02 5.62 NORMAL | | 0 | |
[14] conf=0.61  \866460% % 18.5
[15] conf=0.63  “WBC-LYM% % 59.8
[16] conf=0.62  'WBC-MONO% % 19.6
[17] co

,level,page_num,block_num,par_num,line_num,word_num,text,conf,left,top,width,height,x1,y1,x2,y2
4,5,1,1,1,1,1,Date/Time,87,201,96,441,81,201,96,642,177
5,5,1,1,1,1,2,:,87,681,117,15,48,681,117,696,165
6,5,1,1,1,1,3,2025-01-16,89,765,99,486,66,765,99,1251,165
7,5,1,1,1,1,4,YS,75,1338,96,171,78,1338,96,1509,174
8,5,1,1,1,1,5,1:51:10,96,1554,99,297,66,1554,99,1851,165
9,5,1,1,1,1,6,Performed,96,3645,96,441,69,3645,96,4086,165
10,5,1,1,1,1,7,By,93,4134,99,96,87,4134,99,4230,186
11,5,1,1,1,1,8,[ProCyte_DX],78,4275,99,549,87,4275,99,4824,186
13,5,1,1,1,2,1,Name,96,195,293,239,94,195,293,434,387
14,5,1,1,1,2,2,Unit,96,1461,293,171,94,1461,293,1632,387
